<a href="https://colab.research.google.com/github/ZeleOeO/Artificial-Intelligence-Projects/blob/main/Negative_Positive_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json

In [2]:
import pandas as pd
import numpy as np

import tensorflow
from tensorflow import keras

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

Initialize the tokenizer and read the data from the csv

In [3]:
tokenizer = Tokenizer(oov_token="<OOV_Token>")


dataset = pd.read_csv("/content/data.csv")

sentences = dataset["Sentence"]
labels = dataset["Sentiment"]

print(type(sentences))
print(dataset.shape)

<class 'pandas.core.series.Series'>
(5842, 2)


In [4]:
labels

0       positive
1       negative
2       positive
3        neutral
4        neutral
          ...   
5837    negative
5838     neutral
5839     neutral
5840     neutral
5841    positive
Name: Sentiment, Length: 5842, dtype: object

In [35]:
training_size = 4000
num_epochs = 175
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'

In [6]:
training_sentences = sentences[0:training_size]
training_labels = labels[0:training_size]

testing_sentences = sentences[training_size:]
testing_labels = labels[training_size:]

In [7]:
training_labels

0       positive
1       negative
2       positive
3        neutral
4        neutral
          ...   
3995    negative
3996     neutral
3997     neutral
3998    positive
3999    negative
Name: Sentiment, Length: 4000, dtype: object

In [8]:
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, padding=padding_type, truncating = trunc_type, maxlen=max_length)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, padding=padding_type, truncating = trunc_type, maxlen=max_length)

Convert the labels and the padded texts to a numpy array

In [9]:

testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)


In [10]:
training_labels

array(['positive', 'negative', 'positive', ..., 'neutral', 'positive',
       'negative'], dtype=object)

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()  # Create a LabelEncoder object
le.fit(labels)  # Fit the encoder to the labels
testing_labels = le.transform(testing_labels)  # Convert testing labels to integers
testing_labels = keras.utils.to_categorical(testing_labels, num_classes=3)

training_labels = keras.utils.to_categorical(le.transform(training_labels), num_classes=3)
#training_labels = le.transform(training_labels)

In [ ]:
training_labels[0:100]

In [13]:
training_labels = training_labels.reshape(-1, 3)

Create the model with an embedder and then fit it

In [14]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(24, activation = "relu"),
    keras.layers.Dense(3, activation = "softmax")
])

model.compile(loss="categorical_crossentropy", optimizer = "adam", metrics= keras.metrics.CategoricalAccuracy(name='accuracy'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 3)                 75        
                                                                 
Total params: 160,483
Trainable params: 160,483
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/175
125/125 - 1s - loss: 0.0901 - accuracy: 0.9320 - val_loss: 3.4232 - val_accuracy: 0.6433 - 515ms/epoch - 4ms/step
Epoch 2/175
125/125 - 0s - loss: 0.0900 - accuracy: 0.9277 - val_loss: 3.4807 - val_accuracy: 0.6314 - 492ms/epoch - 4ms/step
Epoch 3/175
125/125 - 0s - loss: 0.0897 - accuracy: 0.9380 - val_loss: 3.4836 - val_accuracy: 0.6433 - 495ms/epoch - 4ms/step
Epoch 4/175
125/125 - 0s - loss: 0.0896 - accuracy: 0.9317 - val_loss: 3.3924 - val_accuracy: 0.6390 - 481ms/epoch - 4ms/step
Epoch 5/175
125/125 - 0s - loss: 0.0903 - accuracy: 0.9340 - val_loss: 3.5182 - val_accuracy: 0.6460 - 460ms/epoch - 4ms/step
Epoch 6/175
125/125 - 0s - loss: 0.0888 - accuracy: 0.9367 - val_loss: 3.6091 - val_accuracy: 0.6477 - 454ms/epoch - 4ms/step
Epoch 7/175
125/125 - 0s - loss: 0.0891 - accuracy: 0.9305 - val_loss: 3.4914 - val_accuracy: 0.6395 - 474ms/epoch - 4ms/step
Epoch 8/175
125/125 - 0s - loss: 0.0901 - accuracy: 0.9243 - val_loss: 3.5747 - val_accuracy: 0.6460 - 492ms/epoch - 4

In [34]:
sentence = [
    "HSBC board recommends shareholders vote against spin-off resolution",
    "China's central bank calls for stronger defences against financial crisis",
    "Shell's $70 Billion BG Deal Meets Shareholder Skepticism",
    "ECB's De Guindos warns of broad risks in financial sector",
    "$ESI on lows, down $1.50 to $2.50 BK a real possibility"
]

sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding = padding_type, truncating=trunc_type)

for index, i in enumerate(model.predict(padded)):
  i = list(i)
  if i.index(max(i)) == 0:
    print(f"'{sentence[index]}' : Negative")
  elif i.index(max(i)) == 1:
    print(f"'{sentence[index]}' : Neutral")
  else:
    print(f"'{sentence[index]}' : Positive")

1/1 [==============================] - 0s 19ms/step
'HSBC board recommends shareholders vote against spin-off resolution' : Neutral
'China's central bank calls for stronger defences against financial crisis' : Negative
'Shell's $70 Billion BG Deal Meets Shareholder Skepticism' : Negative
'ECB's De Guindos warns of broad risks in financial sector' : Neutral
'$ESI on lows, down $1.50 to $2.50 BK a real possibility' : Negative


In [33]:
model.predict(padded)

1/1 [==============================] - 0s 18ms/step


array([[1.3715116e-04, 9.9985647e-01, 6.2864310e-06],
       [9.5621061e-01, 2.7122153e-03, 4.1077085e-02],
       [4.1218314e-07, 9.2735481e-01, 7.2644718e-02],
       [5.3750828e-05, 9.9994516e-01, 1.1108704e-06],
       [9.9996328e-01, 6.5061579e-07, 3.6085636e-05]], dtype=float32)